# 프로젝트 명 : Seoul_Crime

# 1. 프로젝트 개요

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

import socket

from IPython.conftest import get_ipython

hostname = socket.gethostname()

## 2. 데이터 개요
### 1) 데이터 읽기

In [2]:
import pandas as pd
import numpy as np
crime_raw_data=pd.DataFrame()

if "Dongmin" in hostname:
    crime_raw_data=pd.read_csv("../../data/02.Seoul_Crime/02. crime_in_Seoul.csv",thousands=",",encoding="euc-kr")
else :
    crime_raw_data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/02.Seoul_Crime/02. crime_in_Seoul.csv",thousands=",",encoding="euc-kr")

crime_raw_data.head()

,구분,죄종,발생검거,건수
0,중부,살인,발생,2.0
1,중부,살인,검거,2.0
2,중부,강도,발생,3.0
3,중부,강도,검거,3.0
4,중부,강간,발생,141.0


### 2) 데이터 개요 보기
- 문제점 :
  - RangeIndex : 65534
  - Not-null 데이터 : 310
  - null 데이터가 많음

In [3]:
crime_raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65534 entries, 0 to 65533
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   구분      310 non-null    object 
 1   죄종      310 non-null    object 
 2   발생검거    310 non-null    object 
 3   건수      310 non-null    float64
dtypes: float64(1), object(3)
memory usage: 2.0+ MB


### 3) 잘못된 데이터 보기
- 죄종에 어떤 값이 들어가있는지 보기
- null 값에 얼마나 많은 데이터들이 있는지 보기

In [4]:
crime_raw_data["죄종"].unique()

array(['살인', '강도', '강간', '절도', '폭력', nan], dtype=object)

- 310번부터 65534번까지 데이터가 모두 NaN 값임

In [5]:
crime_raw_data[crime_raw_data["죄종"].isnull()]

,구분,죄종,발생검거,건수
310,NaN,NaN,NaN,NaN
311,NaN,NaN,NaN,NaN
312,NaN,NaN,NaN,NaN
313,NaN,NaN,NaN,NaN
314,NaN,NaN,NaN,NaN
...,...,...,...,...
65529,NaN,NaN,NaN,NaN
65530,NaN,NaN,NaN,NaN
65531,NaN,NaN,NaN,NaN
65532,NaN,NaN,NaN,NaN


In [6]:
crime_raw_data=crime_raw_data[crime_raw_data["죄종"].notnull()]

- 새로운 로우 데이터를 보면 정상적으로 보임
  - integer로 보임

In [7]:
crime_raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 310 entries, 0 to 309
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   구분      310 non-null    object 
 1   죄종      310 non-null    object 
 2   발생검거    310 non-null    object 
 3   건수      310 non-null    float64
dtypes: float64(1), object(3)
memory usage: 12.1+ KB


# 3. Pandas Pivot table
- index, coloumn, values, aggfunc로 사용 함

In [8]:
df = pd.DataFrame()
df=pd.read_excel("../../data/02.Seoul_Crime/02. sales-funnel.xlsx", engine="openpyxl")
if "Dongmin" in hostname:
    pass
    #df=pd.read_excel("../../data/02.Seoul_Crime/02. sales-funnel.xlsx", engine="openpyxl")
else :
    df=pd.read_excel("/content/drive/MyDrive/Colab Notebooks/data/02.Seoul_Crime/02. sales-funnel.xlsx")

df.head()

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


#### values 설정

In [9]:
df.pivot_table(index=["Manager", "Rep"], values="Price")

Price
Manager       Rep                        
Debra Henley  Craig Booker   20000.000000
              Daniel Hilton  38333.333333
              John Smith     20000.000000
Fred Anderson Cedric Moss    27500.000000
              Wendy Yule     44250.000000

#### aggfunc 를 통해 계산된 컬럼 출력

In [10]:
df.pivot_table(index=["Manager", "Rep"], values="Price",aggfunc=[np.sum,len])

sum   len
                              Price Price
Manager       Rep                        
Debra Henley  Craig Booker    80000     4
              Daniel Hilton  115000     3
              John Smith      40000     2
Fred Anderson Cedric Moss    110000     4
              Wendy Yule     177000     4

#### columns 설정

In [11]:
df.pivot_table(index=["Manager", "Rep"], values="Price",columns="Product",aggfunc=[np.sum,len])

sum                               len  \
Product                           CPU Maintenance Monitor Software  CPU   
Manager       Rep                                                         
Debra Henley  Craig Booker    65000.0      5000.0     NaN  10000.0  2.0   
              Daniel Hilton  105000.0         NaN     NaN  10000.0  2.0   
              John Smith      35000.0      5000.0     NaN      NaN  1.0   
Fred Anderson Cedric Moss     95000.0      5000.0     NaN  10000.0  2.0   
              Wendy Yule     165000.0      7000.0  5000.0      NaN  2.0   

                                                          
Product                     Maintenance Monitor Software  
Manager       Rep                                         
Debra Henley  Craig Booker          1.0     NaN      1.0  
              Daniel Hilton         NaN     NaN      1.0  
              John Smith            1.0     NaN      NaN  
Fred Anderson Cedric Moss           1.0     NaN      1.0  
              Wendy Yule            1.0     1.0      NaN

#### NaN값 설정 : fill valuem

In [12]:
df.pivot_table(index=["Manager", "Rep"], values="Price",columns="Product",aggfunc=[np.sum,len],fill_value=0)

sum                              len  \
Product                         CPU Maintenance Monitor Software CPU   
Manager       Rep                                                      
Debra Henley  Craig Booker    65000        5000       0    10000   2   
              Daniel Hilton  105000           0       0    10000   2   
              John Smith      35000        5000       0        0   1   
Fred Anderson Cedric Moss     95000        5000       0    10000   2   
              Wendy Yule     165000        7000    5000        0   2   

                                                          
Product                     Maintenance Monitor Software  
Manager       Rep                                         
Debra Henley  Craig Booker            1       0        1  
              Daniel Hilton           0       0        1  
              John Smith              1       0        0  
Fred Anderson Cedric Moss             1       0        1  
              Wendy Yule              1       1        0

#### 2개 이상의 index와 values 설정

In [13]:
df.pivot_table(index=["Manager", "Rep","Product"], values=["Price","Quantity"],aggfunc=[np.sum,len],fill_value=0)

sum            len         
                                          Price Quantity Price Quantity
Manager       Rep           Product                                    
Debra Henley  Craig Booker  CPU           65000        2     2        2
                            Maintenance    5000        2     1        1
                            Software      10000        1     1        1
              Daniel Hilton CPU          105000        4     2        2
                            Software      10000        1     1        1
              John Smith    CPU           35000        1     1        1
                            Maintenance    5000        2     1        1
Fred Anderson Cedric Moss   CPU           95000        3     2        2
                            Maintenance    5000        1     1        1
                            Software      10000        1     1        1
              Wendy Yule    CPU          165000        7     2        2
                            Maintenance    7000        3     1        1
                            Monitor        5000        2     1        1

#### aggfunc 2개 이상 설정

In [14]:
df.pivot_table(
    index=["Manager", "Rep", "Product"],
    values=["Price", "Quantity"],
    aggfunc=[np.sum, np.mean],
    fill_value=0,
    margins=True
)

sum                   mean  \
                                          Price Quantity         Price   
Manager       Rep           Product                                      
Debra Henley  Craig Booker  CPU           65000        2  32500.000000   
                            Maintenance    5000        2   5000.000000   
                            Software      10000        1  10000.000000   
              Daniel Hilton CPU          105000        4  52500.000000   
                            Software      10000        1  10000.000000   
              John Smith    CPU           35000        1  35000.000000   
                            Maintenance    5000        2   5000.000000   
Fred Anderson Cedric Moss   CPU           95000        3  47500.000000   
                            Maintenance    5000        1   5000.000000   
                            Software      10000        1  10000.000000   
              Wendy Yule    CPU          165000        7  82500.000000   
                            Maintenance    7000        3   7000.000000   
                            Monitor        5000        2   5000.000000   
All                                      522000       30  30705.882353   

                                                   
                                         Quantity  
Manager       Rep           Product                
Debra Henley  Craig Booker  CPU          1.000000  
                            Maintenance  2.000000  
                            Software     1.000000  
              Daniel Hilton CPU          2.000000  
                            Software     1.000000  
              John Smith    CPU          1.000000  
                            Maintenance  2.000000  
Fred Anderson Cedric Moss   CPU          1.500000  
                            Maintenance  1.000000  
                            Software     1.000000  
              Wendy Yule    CPU          3.500000  
                            Maintenance  3.000000  
                            Monitor      2.000000  
All                                      1.764706

# 4. 서울시 범죄 현황 데이터 정리

- pivot table로 변경

In [15]:
crime_station = crime_raw_data.pivot_table(
    crime_raw_data,
    index="구분",
    columns=["죄종","발생검거"],
    aggfunc=[np.sum]
)
crime_station.head()

sum                                                              
         건수                                                              
죄종       강간           강도          살인           절도              폭력        
발생검거     검거     발생    검거    발생    검거   발생      검거      발생      검거      발생
구분                                                                       
강남    269.0  339.0  26.0  24.0   3.0  3.0  1129.0  2438.0  2096.0  2336.0
강동    152.0  160.0  13.0  14.0   5.0  4.0   902.0  1754.0  2201.0  2530.0
강북    159.0  217.0   4.0   5.0   6.0  7.0   672.0  1222.0  2482.0  2778.0
강서    239.0  275.0  10.0  10.0  10.0  9.0  1070.0  1952.0  2768.0  3204.0
관악    264.0  322.0  10.0  12.0   7.0  6.0   937.0  2103.0  2707.0  3235.0

- column 데이터 확인

In [16]:
crime_station.columns

MultiIndex([('sum', '건수', '강간', '검거'),
            ('sum', '건수', '강간', '발생'),
            ('sum', '건수', '강도', '검거'),
            ('sum', '건수', '강도', '발생'),
            ('sum', '건수', '살인', '검거'),
            ('sum', '건수', '살인', '발생'),
            ('sum', '건수', '절도', '검거'),
            ('sum', '건수', '절도', '발생'),
            ('sum', '건수', '폭력', '검거'),
            ('sum', '건수', '폭력', '발생')],
           names=[None, None, '죄종', '발생검거'])

- 멀티 인덱스에서 하나의 데이터에 접근하기
- 시리즈로 반환 받음

In [17]:
crime_station['sum', '건수', '폭력', '발생']

구분
강남     2336.0
강동     2530.0
강북     2778.0
강서     3204.0
관악     3235.0
광진     2392.0
구로     3164.0
금천     2193.0
남대문     890.0
노원     2516.0
도봉     1441.0
동대문    2642.0
동작     1661.0
마포     2939.0
방배      500.0
서대문    1976.0
서부     1128.0
서초     1838.0
성동     1619.0
성북     1176.0
송파     3289.0
수서     1666.0
양천     2250.0
영등포    3593.0
용산     2384.0
은평     1536.0
종로     1291.0
종암      932.0
중랑     3164.0
중부     1335.0
혜화      972.0
Name: (sum, 건수, 폭력, 발생), dtype: float64

- droplevel : 다중 컬럼에서 특정컬럼 제거

In [18]:
crime_station.columns = crime_station.columns.droplevel([0,1])

In [19]:
crime_station.columns

MultiIndex([('강간', '검거'),
            ('강간', '발생'),
            ('강도', '검거'),
            ('강도', '발생'),
            ('살인', '검거'),
            ('살인', '발생'),
            ('절도', '검거'),
            ('절도', '발생'),
            ('폭력', '검거'),
            ('폭력', '발생')],
           names=['죄종', '발생검거'])

In [20]:
crime_station.head()

죄종       강간           강도          살인           절도              폭력        
발생검거     검거     발생    검거    발생    검거   발생      검거      발생      검거      발생
구분                                                                       
강남    269.0  339.0  26.0  24.0   3.0  3.0  1129.0  2438.0  2096.0  2336.0
강동    152.0  160.0  13.0  14.0   5.0  4.0   902.0  1754.0  2201.0  2530.0
강북    159.0  217.0   4.0   5.0   6.0  7.0   672.0  1222.0  2482.0  2778.0
강서    239.0  275.0  10.0  10.0  10.0  9.0  1070.0  1952.0  2768.0  3204.0
관악    264.0  322.0  10.0  12.0   7.0  6.0   937.0  2103.0  2707.0  3235.0

- 현재 index= 경찰서 이름
- 경찰서 이름을 통해 지역구 이름 유추해야함

In [21]:
crime_station.index

Index(['강남', '강동', '강북', '강서', '관악', '광진', '구로', '금천', '남대문', '노원', '도봉',
       '동대문', '동작', '마포', '방배', '서대문', '서부', '서초', '성동', '성북', '송파', '수서',
       '양천', '영등포', '용산', '은평', '종로', '종암', '중랑', '중부', '혜화'],
      dtype='object', name='구분')

# 4. Python 모듈 설치

## pip
- 파이썬 공식 모듈 관리자
- 명령어 :
  - pip list : 현재 설치된 모듈 반환
  - pip install 모듈명 : 모듈 설치
  - pip uninstall 모듈명 : 모듈 삭제

## conda
- pip는 콘다 환경에서 의존성 관리가 정확하지 않음
- 다른 명령어는 pip와 동일
- conda install -c 채널명 모듈 : 해당 채널에 모듈 설치
- 단, 모든 모듈이 콘다로 설치되는 것이 아님


In [22]:
get_ipython().system("pip list")

Package                   Version
------------------------- -----------
anyio                     4.2.0
argon2-cffi               21.3.0
argon2-cffi-bindings      21.2.0
asttokens                 2.0.5
async-lru                 2.0.4
attrs                     24.2.0
Babel                     2.11.0
backcall                  0.2.0
beautifulsoup4            4.12.3
bleach                    4.1.0
Bottleneck                1.3.7
Brotli                    1.0.9
certifi                   2024.8.30
cffi                      1.17.1
charset-normalizer        3.3.2
colorama                  0.4.6
comm                      0.2.1
contourpy                 1.0.5
cycler                    0.11.0
debugpy                   1.6.7
decorator                 5.1.1
defusedxml                0.7.1
et-xmlfile                1.1.0
exceptiongroup            1.2.0
executing                 0.8.3
fastjsonschema            2.16.2
fonttools                 4.51.0
h11                       0.14.0
httpcore          

# 5. Google Mpas API 이용

In [6]:
import googlemaps

In [4]:
import json
import os

current_dir = os.getcwd()
config_dir = os.path.join(current_dir, '../../data/')

config_file_path = os.path.join(config_dir, 'config.json')
with open(config_file_path, "r") as f:
    config = json.load(f)
gmaps_key = config["gcp"]["api_key"]

In [7]:
gmaps = googlemaps.Client(key=gmaps_key)

In [8]:
gmaps.geocode("서울영등포경찰서",language="ko")

[{'address_components': [{'long_name': '608',
    'short_name': '608',
    'types': ['premise']},
   {'long_name': '국회대로',
    'short_name': '국회대로',
    'types': ['political', 'sublocality', 'sublocality_level_4']},
   {'long_name': '영등포구',
    'short_name': '영등포구',
    'types': ['political', 'sublocality', 'sublocality_level_1']},
   {'long_name': '서울특별시',
    'short_name': '서울특별시',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': '대한민국',
    'short_name': 'KR',
    'types': ['country', 'political']},
   {'long_name': '150-043',
    'short_name': '150-043',
    'types': ['postal_code']}],
  'formatted_address': '대한민국 서울특별시 영등포구 국회대로 608',
  'geometry': {'location': {'lat': 37.5260441, 'lng': 126.9008091},
   'location_type': 'ROOFTOP',
   'viewport': {'northeast': {'lat': 37.5273930802915,
     'lng': 126.9021580802915},
    'southwest': {'lat': 37.5246951197085, 'lng': 126.8994601197085}}},
  'place_id': 'ChIJ1TimJLaffDURptXOs0Tj6sY',
  'plus_code': {'comp